In [ ]:
!pip install numpy==1.26.0 scipy==1.14.0 meshio==5.3.5 libigl==v2.5.1 polyscope==2.2.1 ilupp==1.0.2 ipctk==1.2.0 pbatoolkit

In [ ]:
import pbatoolkit as pbat
import pbatoolkit.geometry
import pbatoolkit.profiling
import pbatoolkit.math.linalg
import meshio
import numpy as np
import scipy as sp
import polyscope as ps
import polyscope.imgui as imgui
import math
import argparse
import os

# Define the input paths and other parameters

In [ ]:
current_path = os.getcwd()

input_path = os.path.join(current_path, 'python' ,'examples', 'notebooks', 'resources', 'bunny.obj')

refined_input_path = os.path.join(current_path, 'python', 'examples', 'notebooks', 'resources', 'bunny_refined.obj')

rho = 1000.
Y = 1e6
nu = 0.45

# Load domain and visual meshes

In [ ]:

imesh = meshio.read(input_path)
V, C = imesh.points, imesh.cells_dict["tetra"]
V = V.astype(np.float64, order='c')
C = C.astype(np.int64, order='c')
rimesh = meshio.read(refined_input_path)
VR, FR = rimesh.points, rimesh.cells_dict["triangle"]
VR = VR.astype(np.float64, order='c')
FR = FR.astype np.int64, order='c')

# Construct FEM quantities for simulation

In [ ]:
mesh = pbat.fem.Mesh(
    V.T, C.T, element=pbat.fem.Element.Tetrahedron, order=2)
x = mesh.X.reshape(math.prod(mesh.X.shape), order='f')
v = np.zeros(x.shape[0])
detJeM = pbat.fem.jacobian_determinants(mesh, quadrature_order=4)
M = pbat.fem.MassMatrix(mesh, detJeM, rho=rho,
                         dims=3, quadrature_order=4).to_matrix()
Minv = pbat.math.linalg.ldlt(M)
Minv.compute(M)

# Construct load vector from gravity field

In [ ]:
g = np.zeros(mesh.dims)
g[-1] = -9.81
detJef = pbat.fem.jacobian_determinants(mesh, quadrature_order=2)
nquadpts = mesh.E.shape[1] * mesh.quadrature_weights(2).shape[0]
fe = np.tile(rho*g[:, np.newaxis], nquadpts)
qgf = pbat.fem.inner_product_weights(
    mesh, quadrature_order=2).flatten(order="F")
Qf = sp.sparse.diags_array([qgf], offsets=[0])
Nf = pbat.fem.shape_function_matrix(mesh, quadrature_order=2)
f = fe @ Qf @ Nf
f = f.reshape(math.prod(f.shape), order="F")
a = Minv.solve(f).squeeze()

# Create hyper elastic potential

In [ ]:
detJeU = pbat.fem.jacobian_determinants(mesh, quadrature_order=4)
GNeU = pbat.fem.shape_function_gradients(mesh, quadrature_order=4)
Y = np.full(mesh.E.shape[1], Y)
nu = np.full(mesh.E.shape[1], nu)
psi = pbat.fem.HyperElasticEnergy.StableNeoHookean
hep = pbat.fem.HyperElasticPotential(
    mesh, detJeU, GNeU, Y, nu, energy=psi, quadrature_order=4)
hep.precompute_hessian_sparsity()

# Set Dirichlet boundary conditions

In [ ]:
Xmin = mesh.X.min(axis=1)
Xmax = mesh.X.max(axis=1)
Xmax[0] = Xmin[0]+1e-4
Xmin[0] = Xmin[0]-1e-4
aabb = pbat.geometry.aabb(np.vstack((Xmin, Xmax)).T)
vdbc = aabb.contained(mesh.X)
dbcs = np.array(vdbc)[:, np.newaxis]
dbcs = np.repeat(dbcs, mesh.dims, axis=1)
for d in range(mesh.dims):
    dbcs[:, d] = mesh.dims*dbcs[:, d]+d
dbcs = dbcs.reshape(math.prod(dbcs.shape))
n = x.shape[0]
dofs = np.setdiff1d(list(range(n)), dbcs)

# Setup linear solver

In [ ]:
Hdd = hep.to_matrix()[:, dofs].tocsr()[dofs, :]
Hddinv = pbat.math.linalg.ldlt(Hdd)
Hddinv.analyze(Hdd)

# Setup visual mesh

In [ ]:
bvh = pbat.geometry.bvh(V.T, C.T, cell=pbat.geometry.Cell.Tetrahedron)
e = bvh.nearest_primitives_to_points(VR.T)
Xi = pbat.fem.reference_positions(mesh, e, VR.T)
phi = pbat.fem.shape_functions_at(mesh, Xi)

ps.set_verbosity(0)
ps.set_up_dir("z_up")
ps.set_front_dir("neg_y_front")
ps.set_ground_plane_mode("shadow_only")
ps.set_ground_plane_height_factor(0.5)
ps.set_program_name("Higher Order Elasticity")
ps.init()
vm = ps.register_volume_mesh("Domain", V, C)
sm = ps.register_surface_mesh("Visual", VR, FR)
pc = ps.register_point_cloud("Dirichlet", mesh.X[:, vdbc].T)
dt = 0.01
animate = False
dx = np.zeros(n)

profiler = pbat.profiling.Profiler()

In [ ]:
def callback():
    global x, v, dx, hep, dt, M, Minv, f, animate, step, profiler
    changed, dt = imgui.InputFloat("dt", dt)
    changed, animate = imgui.Checkbox("animate", animate)
    step = imgui.Button("step")

    if animate or step:
        profiler.begin_frame("Physics")
        # 1 Newton step
        hep.compute_element_elasticity(x, grad=True, hessian=True)
        gradU, HU = hep.gradient(), hep.hessian()
        dt2 = dt**2
        xtilde = x + dt*v + dt2*a
        A = M + dt2 * HU
        b = -(M @ (x - xtilde) + dt2*gradU)
        Add = A.tocsc()[:, dofs].tocsr()[dofs, :]
        bd = b[dofs]
        Hddinv.factorize(Add)
        dx[dofs] = Hddinv.solve(bd).squeeze()
        v = dx / dt
        x = x + dx
        profiler.end_frame("Physics")

        # Update visuals
        X = x.reshape(mesh.X.shape[0], mesh.X.shape[1], order='f')
        xv = (X[:,mesh.E[:,e]] * phi).sum(axis=1)
        sm.update_vertex_positions(xv.T)

ps.set_user_callback(callback)
ps.show()